In [ ]:
import cv2
import os
import numpy as np
import tensorrt as trt
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import pycuda.driver as cuda
import pycuda.autoinit

In [ ]:

TRT_LOGGER = trt.Logger()

# ----------- 엔진 로드 -----------
with open("best.trt", "rb") as f:
    runtime = trt.Runtime(TRT_LOGGER)
    engine = runtime.deserialize_cuda_engine(f.read())
    context = engine.create_execution_context()


In [ ]:

# ----------- 추론 함수 -----------
def infer_one_image(img_path):
    img = cv2.imread(img_path)
    img_resized = cv2.resize(img, (640, 640))
    img_input = img_resized.transpose(2, 0, 1).astype(np.float32) / 255.0
    img_input = np.expand_dims(img_input, axis=0)
    img_input.astype(np.float32)

    # transform to C-order
    img_input = np.ascontiguousarray(img_input, dtype=np.float32)
    # transform to F-order
    #img_input = np.asfortranarray(img_input, dtype=np.float32)
    #print(img_input.flags)

    d_input = cuda.mem_alloc(img_input.nbytes)
    output = np.empty(engine.get_binding_shape(1), dtype=np.float32)
    d_output = cuda.mem_alloc(output.nbytes)
    bindings = [int(d_input), int(d_output)]

    cuda.memcpy_htod(d_input, img_input)
    context.execute_v2(bindings)
    cuda.memcpy_dtoh(output, d_output)

    #print(f"{os.path.basename(img_path)} → 추론 결과 shape:", output.shape)
    # 필요시 output값 리턴
    return output, img_resized


In [ ]:

# ----------- 평가용 유틸 함수 -----------
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def xywh2xyxy(box):
    x, y, w, h = box[:4]
    x1 = x - w/2
    y1 = y - h/2
    x2 = x + w/2
    y2 = y + h/2
    return [x1, y1, x2, y2]

def nms(boxes, iou_thres):
    if len(boxes) == 0:
        return []

    boxes = np.array(boxes)  # 리스트라면 배열로 변환
    scores = boxes[:, 4]     # confidence score 추출
    order = scores.argsort()[::-1]  # 높은 점수 순 정렬

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)

        if order.size == 1:
            break

        candidates = boxes[order[1:]]  # 다음 후보들
        if candidates.ndim == 1:
            candidates = candidates.reshape(1, -1)  # 안전하게 reshape

        ious = compute_ious(boxes[i], candidates)
        if ious.size == 0:
            break

        inds = np.where(ious < iou_thres)[0]
        order = order[inds + 1]  # IoU 낮은 것만 남김
    return keep

def compute_ious(box, boxes):
    if boxes.ndim == 1:
        boxes = boxes.reshape(1,-1)
    box = xywh2xyxy(box)
    boxes = np.array([xywh2xyxy(b) for b in boxes])
    xi1 = np.maximum(box[0], boxes[:,0])
    yi1 = np.maximum(box[1], boxes[:,1])
    xi2 = np.minimum(box[2], boxes[:,2])
    yi2 = np.minimum(box[3], boxes[:,3])
    inter_area = np.maximum(xi2-xi1,0) * np.maximum(yi2-yi1,0)
    box_area = (box[2]-box[0])*(box[3]-box[1])
    boxes_area = (boxes[:,2]-boxes[:,0])*(boxes[:,3]-boxes[:,1])
    union_area = box_area + boxes_area - inter_area
    iou = inter_area / (union_area + 1e-6)
    return iou

def yolo_head_process(output, conf_thres=0.5, iou_thres=0.5):
    output = output.squeeze(0)  # (5, 8400)
    output = output.T           # (8400, 5)
    print('transformed output=',output.shape)

    boxes = []
    for row in output:
        #print('row=', row.shape)
        #obj_conf = sigmoid(row[4]) # 마지막 값은 object confidence
        score = sigmoid(row[4])  # 클래스가 하나라 class score는 곧 obj_conf

        if score > conf_thres:
            x, y, w, h = row[:4]
            boxes.append([x, y, w, h, score, 0]) #class_id is always 0

    #boxes = np.array(boxes)

    boxes = np.array(boxes,dtype=np.float32).reshape(-1,6)
    print("boxes shape = ",boxes.shape)

    if len(boxes) == 0:
        return []

    print(len(boxes))

    keep = nms(boxes, iou_thres)
    return boxes[keep]

def load_yolo_label(label_path, img_shape):
    gt_boxes = []
    h, w = img_shape[:2]
    with open(label_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            class_id = int(parts[0])
            cx, cy, bw, bh = map(float, parts[1:])
            x = cx * w
            y = cy * h
            bw = bw * w
            bh = bh * h
            gt_boxes.append([x, y, bw, bh, class_id])
    return gt_boxes

def eval_one_image(pred_boxes, gt_boxes, iou_thres=0.5):
    gt_matched = [False] * len(gt_boxes)
    tp, fp = [], []
    for pb in sorted(pred_boxes, key=lambda x: -x[4]):
        found = False
        for i, gb in enumerate(gt_boxes):
            if pb[5] == gb[4]:
                iou = compute_ious(pb, np.array([gb]))[0]
                if iou > iou_thres and not gt_matched[i]:
                    tp.append(1)
                    fp.append(0)
                    gt_matched[i] = True
                    found = True
                    break
        if not found:
            tp.append(0)
            fp.append(1)
    return tp, fp, len(gt_boxes)

def compute_ap(tp, fp, num_gt):
    tp = np.cumsum(tp)
    fp = np.cumsum(fp)
    recall = tp / (num_gt + 1e-8)
    precision = tp / (tp + fp + 1e-8)
    ap = np.trapz(precision, recall)
    return ap, precision[-1] if len(precision)>0 else 0, recall[-1] if len(recall)>0 else 0


In [ ]:

# ----------- 실제 평가 루프 -----------
image_folder = "/home/seungsun/project/val_dataset/valid/images"
image_files = [os.path.join(image_folder, fname)
               for fname in os.listdir(image_folder)
               if fname.lower().endswith(('.jpg', '.jpeg', '.png'))]

all_tp, all_fp, all_gt = [], [], 0

#for img_path in image_files:
for img_path in tqdm(image_files, desc="Evaluating Images", unit="img"):
    output, img_resized = infer_one_image(img_path)
    print(output)
    print(output.shape)
    pred_boxes = yolo_head_process(output)
    label_path = img_path.replace('/images/', '/labels/').rsplit('.', 1)[0] + '.txt'
    if not os.path.exists(label_path):
        continue
    gt_boxes = load_yolo_label(label_path, img_resized.shape)
    tp, fp, num_gt = eval_one_image(pred_boxes, gt_boxes)
    all_tp.extend(tp)
    all_fp.extend(fp)
    all_gt += num_gt

    #memory manage
    del output, img_resized, pred_boxes, gt_boxes
    gc.collect()

ap, prec, rec = compute_ap(np.array(all_tp), np.array(all_fp), all_gt)
print(f"mAP(근사): {ap:.4f} | Precision: {prec:.4f} | Recall: {rec:.4f}")


In [ ]:

# ----------- 성능 평가 저장 -----------
result_path = "/home/seungsun/project/eval_results.csv"
pd.DataFrame({
    "AP": [ap],
    "Precision": [prec],
    "Recall": [rec],
}).to_csv(result_path, index=False)


In [ ]:

# ----------- 성능 시각화 저장 -----------
plt.figure(figsize=(6,4))
plt.bar(["AP", "Precision", "Recall"], [ap, prec, rec], color="skyblue")
plt.title("Detection Evaluation Metrics")
plt.ylim(0, 1)
plt.savefig("/home/seungsun/project/eval_barplot.png")
plt.close()
